In [1]:
import os
from glob import glob
# from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, Wav2Vec2Processor
import torchaudio
import soundfile as sf
import librosa
import torch.nn.functional as F
import torch.nn as nn
from IPython.display import Audio, display, Markdown
import torch

import pandas as pd
import random

In [2]:
# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [3]:
metadata = {
    "angry": sorted(glob(r"data/audio-emotions/Angry/*.wav")),
    "sad": sorted(glob(r"data/audio-emotions/Sad/*.wav")),
    "disgusted": sorted(glob(r"data/audio-emotions/Disgusted/*.wav")),
    "fearful": sorted(glob(r"data/audio-emotions/Fearful/*.wav")),
    "happy": sorted(glob(r"data/audio-emotions/Happy/*.wav")),
    "neutral": sorted(glob(r"data/audio-emotions/Neutral/*.wav")),
    "surprised": sorted(glob(r"data/audio-emotions/Surprised/*.wav")),
}

TRAIN_SPLIT = 0.8

train_meta = {
    "angry": metadata["angry"][:int(TRAIN_SPLIT * len(metadata["angry"]))],
    "sad": metadata["sad"][:int(TRAIN_SPLIT * len(metadata["sad"]))],
    "disgusted": metadata["disgusted"][:int(TRAIN_SPLIT * len(metadata["disgusted"]))],
    "fearful": metadata["fearful"][:int(TRAIN_SPLIT * len(metadata["fearful"]))],
    "happy": metadata["happy"][:int(TRAIN_SPLIT * len(metadata["happy"]))],
    "neutral": metadata["neutral"][:int(TRAIN_SPLIT * len(metadata["neutral"]))],
    "surprised": metadata["surprised"][:int(TRAIN_SPLIT * len(metadata["surprised"]))],
}

test_meta = {
    "angry": metadata["angry"][int(TRAIN_SPLIT * len(metadata["angry"])):],
    "sad": metadata["sad"][int(TRAIN_SPLIT * len(metadata["sad"])):],
    "disgusted": metadata["disgusted"][int(TRAIN_SPLIT * len(metadata["disgusted"])):],
    "fearful": metadata["fearful"][int(TRAIN_SPLIT * len(metadata["fearful"])):],
    "happy": metadata["happy"][int(TRAIN_SPLIT * len(metadata["happy"])):],
    "neutral": metadata["neutral"][int(TRAIN_SPLIT * len(metadata["neutral"])):],
    "surprised": metadata["surprised"][int(TRAIN_SPLIT * len(metadata["surprised"])):],
}
        

In [4]:
train_df = []
test_df = []

for emotion in train_meta:
    for file in train_meta[emotion]:
        train_df.append((file, emotion))
        
for emotion in test_meta:
    for file in test_meta[emotion]:
        test_df.append((file, emotion))
        
random.shuffle(train_df)
random.shuffle(test_df)


In [5]:
# create pandas dataframes
train_df = pd.DataFrame(train_df, columns=["file", "emotion"])
test_df = pd.DataFrame(test_df, columns=["file", "emotion"])

# apply feature extractor to all files
# train_df = train_df.head()
# test_df = test_df.head()
# flattener = nn.Flatten()
def extract_features(file):
    # waveform, sample_rate = sf.read(file)
    waveform, sample_rate = librosa.load(file, sr=16000)
    # input_values = tokenizer(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values
    # sampling_rate=sample_rate
    features = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="np").input_values
    
    # features = features[0]
    print(f"{features.shape = }")
    return features.tolist()


In [6]:

train_df = train_df.head()
test_df = test_df.head()
train_df["features"] = train_df["file"].apply(extract_features)
test_df["features"] = test_df["file"].apply(extract_features)
train_df.head()

features.shape = (1, 72606)
features.shape = (1, 74208)
features.shape = (1, 49650)
features.shape = (1, 37904)
features.shape = (1, 71538)
features.shape = (1, 67109)
features.shape = (1, 43306)
features.shape = (1, 27706)
features.shape = (1, 29135)
features.shape = (1, 38306)


,file,emotion,features
0,data/audio-emotions/Disgusted/03-01-07-02-01-0...,disgusted,"[[-0.0010183249833062291, -0.00094243895728141..."
1,data/audio-emotions/Angry/03-02-05-02-02-02-23...,angry,"[[-0.0016397269209846854, -0.00294094602577388..."
2,data/audio-emotions/Disgusted/1073_DFA_DIS_XX.wav,disgusted,"[[0.009281053207814693, 0.01408199779689312, 0..."
3,data/audio-emotions/Fearful/1054_IEO_FEA_MD.wav,fearful,"[[1.1033457517623901, 1.2063554525375366, 1.16..."
4,data/audio-emotions/Happy/03-02-03-01-01-02-12...,happy,"[[5.423857510322705e-05, 5.423857510322705e-05..."


In [7]:
# transcription
def transcribe(features):
    input_values = torch.tensor(features)
    logits = model(input_values).logits[0]
    predicted_ids = torch.argmax(logits, dim=-1)
    outputs = tokenizer.decode(predicted_ids, output_word_offsets=True)
    time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate
    
    word_offsets = [
        { "word": d["word"],
            "start_time": round(d["start_offset"] * time_offset, 2),
            "end_time": round(d["end_offset"] * time_offset, 2),
        }
        for d in outputs.word_offsets
    ]

    return word_offsets

In [8]:
predictions_df = train_df.copy()
predictions_df["transcription"] = predictions_df["features"].apply(transcribe)
predictions_df.head()

,file,emotion,features,transcription
0,data/audio-emotions/Disgusted/03-01-07-02-01-0...,disgusted,"[[-0.0010183249833062291, -0.00094243895728141...","[{'word': 'KIDS', 'start_time': 1.1, 'end_time..."
1,data/audio-emotions/Angry/03-02-05-02-02-02-23...,angry,"[[-0.0016397269209846854, -0.00294094602577388...","[{'word': 'DOGS', 'start_time': 1.02, 'end_tim..."
2,data/audio-emotions/Disgusted/1073_DFA_DIS_XX.wav,disgusted,"[[0.009281053207814693, 0.01408199779689312, 0...","[{'word': 'DON'T', 'start_time': 1.32, 'end_ti..."
3,data/audio-emotions/Fearful/1054_IEO_FEA_MD.wav,fearful,"[[1.1033457517623901, 1.2063554525375366, 1.16...","[{'word': 'IT'S', 'start_time': 0.74, 'end_tim..."
4,data/audio-emotions/Happy/03-02-03-01-01-02-12...,happy,"[[5.423857510322705e-05, 5.423857510322705e-05...","[{'word': 'KIDS', 'start_time': 1.04, 'end_tim..."


In [9]:
print(f"TRANSCRIPTION: {' '.join([ item['word'] for item in predictions_df['transcription'].iloc[2]])}")
Audio(train_df["file"].iloc[2], autoplay=True)

TRANSCRIPTION: DON'T FORGET A JACKET


In [10]:
train_df["emotion"].iloc[2]

'disgusted'

In [19]:
# train_df.iloc[0]["features"]

In [20]:
# save to csv
# train_df.to_csv("data/audio_emotions-train.csv", index=False)
# test_df.to_csv("data/audio_emotions-test.csv", index=False)

---

## Trying to use loading script to load entire dataset

In [1]:
from local_dataset import AudioEmotionsDataset
from IPython.display import Audio, display, Markdown

In [7]:
# NOTE: remove max_size to load entire dataset.
local_dataset = AudioEmotionsDataset("data/audio-emotions", max_size=10)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [8]:
waveform, X, y = local_dataset.get_train(0)
waveform.shape, X.shape, y

((56590,), torch.Size([1, 56590]), 4)

In [9]:
waveform[0], X[0][0]

(0.0, tensor(0.0007))

In [10]:
_, transcription = local_dataset.transcribe(X)

print(f"TRANSCRIPTION: {transcription}")
Audio(waveform,  rate=16000, autoplay=True)

TRANSCRIPTION: KIDS ARE TALKING BY THE DOOR


In [11]:
encoded = local_dataset.one_hot_encode(y)
label = local_dataset.num_to_label(y)
encoded, label

(tensor([0, 0, 0, 0, 1, 0, 0]), 'happy')